### IMDB 영화평 감성분석 (이진분류)

- CountVectorizer + LogisticRegression

1. 데이터 탐색

In [62]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [63]:
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t', quoting=3)      # quoting=3 -> QUOTE NONE
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [65]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

2. 텍스트 전처리

In [66]:
# <br> 태그 공백으로 변환
df.review = df.review.str.replace('<br />', ' ')

In [67]:
# 구둣점, 숫자 제거 --> 영어 이외의 문자 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]', ' ')

In [68]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

3. 데이터셋 분리

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review.values, df.sentiment.values, stratify=df.sentiment.values, test_size=0.2, random_state=2023
)

In [70]:
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([10000, 10000], dtype=int64))

4. Text encoding

In [71]:
# 아래와 같은 방법으로 하면 train <-> test 데이터 간 매핑오류 발생가능
cvect.fit_transform(X_train).shape, cvect.fit_transform(X_test).shape

((20000, 66602), (5000, 37763))

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((20000, 66602), (5000, 66602))

5. 학습 및 평가

In [73]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2023, max_iter=500)

In [74]:
# 시간이 오래 걸리는 작업 - %time : magic명령어 사용
%time lrc.fit(X_train_cv, y_train)

CPU times: total: 17.7 s
Wall time: 5.23 s


LogisticRegression(max_iter=500, random_state=2023)

In [75]:
lrc.score(X_test_cv, y_test)

0.8786

6. Bigram

In [76]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1, 2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((20000, 1455899), (5000, 1455899))

In [88]:
lrc2 = LogisticRegression(random_state=2023, max_iter=500)
%time lrc2.fit(X_train_cv2, y_train)

CPU times: total: 2min 31s
Wall time: 47.1 s


LogisticRegression(max_iter=500, random_state=2023)

In [89]:
lrc2.score(X_test_cv2, y_test)

0.8896

7. 모델 save / load

In [90]:
import joblib

In [91]:
# 모델 저장
joblib.dump(cvect2, 'model/imdb_cvect_2.pkl')
joblib.dump(lrc2, 'model/imdb_lrc2.pkl')

['model/imdb_lrc2.pkl']

In [93]:
# 모델 로드
new_cvect = joblib.load('model/imdb_cvect_2.pkl')
new_lrc = joblib.load('model/imdb_lrc2.pkl')

8. 실제 데이터로 검증

In [105]:
review = ['''
This isn't just a beautifully crafted gangster film. Or an outstanding family portrait, for that matter. An amazing period piece. A character study. A lesson in filmmaking and an inspiration to generations of actors, directors, screenwriters and producers. For me, this is more: this is the definitive film. 10 stars out of 10.
''', 
'''I continually fail to understand why The Godfather is hailed as "The Greatest Movie of All Time". I've seen it twice--a second time just to make sure--and I have to tell you that I sat there in a stupor, bored out of my mind. And I'm not a teenager raised on MTV; I'm in my 30s and am absolutely devoted to movies--I've seen as many classics (American & foreign) that I can get my hands on. But, for me, The Godfather ranks alongside Singin' in the Rain as the most overrated films of all time.
Singin' in the Rain, at least, I get (it's just my intense dislike for Donald O'Connor that makes me dislike this film). But The Godfather? It's just a bland epic about a bunch of moronic gangsters, with Marlon Brando giving a campy performance, and riddled with repulsive violence. Give me a break. The fact that this movie is so "beloved" has had the direct result that nowadays we got absurdly worse and worse films every year, created by clueless filmmakers.''',
'''"The Godfather" was a sickening experience the first time I saw it in its initial release, and it hasn't changed. I thoroughly disliked the film's spirit, finding it very ugly at its heart. I saw little justification for its existence, was sorry I saw it, and have tried to forget it.''']


In [106]:
# 텍스트 전처리
import re
review = map(lambda x: re.sub('[^A-Za-z]', ' ', x), review)

In [107]:
# feature 변환
review_cv = new_cvect.transform(review)
review_cv.shape

(3, 1455899)

In [108]:
# 예측
for i in new_lrc.predict(review_cv):
    print('긍정' if i == 1 else '부정')

긍정
부정
긍정
